# Build an agent with memory

Create an AI agent that remembers important information across conversations.

## Problem

You want to build an AI agent that can store and recall important information—user preferences, key facts, or context from previous conversations.

| Memory type | Example |
|-------------|---------|
| User preferences | "I prefer Python over JavaScript" |
| Key facts | "The project deadline is March 15" |
| Conversation context | "We discussed the budget yesterday" |

## Solution

**What's in this recipe:**
- Store memories with embeddings for semantic search
- Retrieve relevant memories based on conversation context
- Use `@pxt.query` for retrieval functions

This pattern is inspired by [Pixelbot](https://github.com/pixeltable/pixelbot) and [Pixelmemory](https://github.com/pixeltable/pixelmemory).

### Setup

In [ ]:
%pip install -qU pixeltable openai

In [ ]:
import os
import getpass
from datetime import datetime

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')

In [ ]:
import pixeltable as pxt
from pixeltable.functions.openai import embeddings, chat_completions

In [ ]:
# Create a fresh directory
pxt.drop_dir('agent_demo', force=True)
pxt.create_dir('agent_demo')

### Create memory bank

In [ ]:
# Create memory bank table
memories = pxt.create_table(
    'agent_demo.memories',
    {
        'content': pxt.String,           # The memory content
        'category': pxt.String,          # Optional category (preference, fact, etc.)
        'created_at': pxt.Timestamp,     # When the memory was stored
    }
)

In [ ]:
# Add embedding for semantic search
memories.add_computed_column(
    embedding=embeddings(memories.content, model='text-embedding-3-small')
)

# Create index for fast retrieval
memories.add_embedding_index('embedding', metric='cosine')

### Define retrieval function

In [ ]:
# Define a query function to retrieve relevant memories
@pxt.query
def recall_memories(context: str, top_k: int = 3) -> list[dict]:
    """Retrieve memories relevant to the current context."""
    results = memories.order_by(
        memories.embedding.similarity(context),
        asc=False
    ).limit(top_k).select(
        memories.content,
        memories.category
    ).collect()
    return [{'content': r['content'], 'category': r['category']} for r in results]

### Store some memories

In [ ]:
# Store some initial memories
initial_memories = [
    {'content': 'User prefers Python for data analysis', 'category': 'preference', 'created_at': datetime.now()},
    {'content': 'The project deadline is March 15, 2024', 'category': 'fact', 'created_at': datetime.now()},
    {'content': 'User works at a startup in San Francisco', 'category': 'fact', 'created_at': datetime.now()},
    {'content': 'Budget for the ML project is $50,000', 'category': 'fact', 'created_at': datetime.now()},
    {'content': 'User prefers concise explanations over detailed ones', 'category': 'preference', 'created_at': datetime.now()},
]

memories.insert(initial_memories)

### Create conversation table with memory retrieval

In [ ]:
# Create conversation table
conversations = pxt.create_table(
    'agent_demo.conversations',
    {'user_message': pxt.String}
)

In [ ]:
# Add memory retrieval step
conversations.add_computed_column(
    relevant_memories=recall_memories(conversations.user_message, top_k=3)
)

In [ ]:
# Build prompt with memories
@pxt.udf
def build_memory_prompt(user_message: str, relevant_memories: list[dict]) -> str:
    memory_text = '\n'.join([f"- {m['content']}" for m in relevant_memories])
    return f"""You are a helpful assistant with access to the following memories about the user:

{memory_text}

Use these memories to personalize your response when relevant.

User: {user_message}
Assistant:"""

conversations.add_computed_column(
    prompt=build_memory_prompt(conversations.user_message, conversations.relevant_memories)
)

In [ ]:
# Generate response with memory context
conversations.add_computed_column(
    response=chat_completions(
        messages=[{'role': 'user', 'content': conversations.prompt}],
        model='gpt-4o-mini'
    )
)
conversations.add_computed_column(
    assistant_reply=conversations.response.choices[0].message.content
)

### Chat with memory-aware agent

In [ ]:
# Test the memory-aware agent
test_messages = [
    {'user_message': 'What programming language should I use for this project?'},
    {'user_message': 'When do I need to finish this?'},
    {'user_message': 'How much can I spend on cloud resources?'},
]

conversations.insert(test_messages)

In [ ]:
# View conversations with memory
conversations.select(
    conversations.user_message,
    conversations.relevant_memories,
    conversations.assistant_reply
).collect()

## Explanation

**Memory-aware agent architecture:**

```
User Message → Retrieve Memories → Build Prompt → LLM Response
                    ↓
            Memory Bank (with embeddings)
```

**Key components:**

| Component | Purpose |
|-----------|---------|
| Memory table | Store facts, preferences, context |
| Embedding index | Enable semantic memory search |
| `@pxt.query` | Retrieval function for memories |
| Prompt builder | Inject memories into LLM context |

**Adding new memories:**

```python
memories.insert([{
    'content': 'New information to remember',
    'category': 'fact',
    'created_at': datetime.now()
}])
```

## See also

- [Build a RAG pipeline](https://docs.pixeltable.com/howto/cookbooks/agents/pattern-rag-pipeline) - Document retrieval
- [Use tool calling](https://docs.pixeltable.com/howto/cookbooks/agents/llm-tool-calling) - Function calling with LLMs
- [Pixelbot](https://github.com/pixeltable/pixelbot) - Full agent implementation